### Create large matrixs

This notebook computes ISC per spot/condition, and collects corresponding memory data (recognition, recall). All is saved into a large structure that is then read into R.

In [1]:
#note: I manually deleted those few data files for which there are problems with the pupil recording (mostly sub012 and sub055) -- this could be automated.

In [2]:
import os, glob, warnings, re, shutil
import numpy as np
import pandas as pd
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.stats import zscore
from scipy.stats import ttest_1samp
from scipy.stats import pearsonr
from scipy.spatial.distance import squareform

In [3]:

def pupil_isc(data, pairwise=True, summary_statistic=np.mean, verbose=True):
    """Intersubject correlation
    For each voxel or ROI, compute the Pearson correlation between each
    subject's response time series and other subjects' response time series.
    If pairwise is False (default), use the leave-one-out approach, where
    correlation is computed between each subject and the average of the other
    subjects. If pairwise is True, compute correlations between all pairs of
    subjects. If summary_statistic is None, return N ISC values for N subjects
    (leave-one-out) or N(N-1)/2 ISC values for each pair of N subjects,
    corresponding to the upper triangle of the pairwise correlation matrix
    (see scipy.spatial.distance.squareform). Alternatively, supply either
    np.mean or np.median to compute summary statistic of ISCs (Fisher Z will
    be applied and inverted if using mean). Input data should be a list 
    where each item is a time-points by voxels ndarray for a given subject.
    Multiple input ndarrays must be the same shape. If a single ndarray is
    supplied, the last dimension is assumed to correspond to subjects. If 
    only two subjects are supplied, simply compute Pearson correlation
    (precludes averaging in leave-one-out approach, and does not apply
    summary statistic.) Output is an ndarray where the first dimension is
    the number of subjects or pairs and the second dimension is the number
    of voxels (or ROIs).
        
    The implementation is based on the following publication:
    
    .. [Hasson2004] "Intersubject synchronization of cortical activity 
    during natural vision.", U. Hasson, Y. Nir, I. Levy, G. Fuhrmann,
    R. Malach, 2004, Science, 303, 1634-1640.
    Parameters
    ----------
    data : list or ndarray
        fMRI data for which to compute ISC
        
    pairwise : bool, default: False
        Whether to use pairwise (True) or leave-one-out (False) approach
        
    summary_statistic : None
        Return all ISCs or collapse using np.mean or np.median
    Returns
    -------
    iscs : subjects or pairs by voxels ndarray
        ISC for each subject or pair (or summary statistic) per voxel
    """
    
    # Convert list input to 3d and check shapes
    if type(data) == list:
        data_shape = data[0].shape
        for i, d in enumerate(data):
            if d.shape != data_shape:
                raise ValueError("All ndarrays in input list "
                                 "must be the same shape!")
            if d.ndim == 1:
                data[i] = d[:, np.newaxis]
        data = np.dstack(data)

    # Convert input ndarray to 3d and check shape
    elif type(data) == np.ndarray:
        if data.ndim == 2:
            data = data[:, np.newaxis, :]            
        elif data.ndim == 3:
            pass
        else:
            raise ValueError("Input ndarray should have 2 "
                             f"or 3 dimensions (got {data.ndim})!")

    # Infer subjects, TRs, voxels and print for user to check
    n_subjects = data.shape[2]
    n_TRs = data.shape[0]
    n_voxels = data.shape[1]
    if verbose:
        print(f"Assuming {n_subjects} subjects with {n_TRs} time points "
              f"and {n_voxels} voxel(s) or ROI(s).")
    
    # Loop over each voxel or ROI
    voxel_iscs = []
    for v in np.arange(n_voxels):
        voxel_data = data[:, v, :].T
        if n_subjects == 2:
            iscs = pearsonr(voxel_data[0, :], voxel_data[1, :])[0]
            summary_statistic = None
            if verbose:
                print("Only two subjects! Simply computing Pearson correlation.")
        elif pairwise:
            iscs = squareform(np.corrcoef(voxel_data), checks=False)
        elif not pairwise:
            iscs = np.array([pearsonr(subject,
                                      np.mean(np.delete(voxel_data,
                                                        s, axis=0),
                                              axis=0))[0]
                    for s, subject in enumerate(voxel_data)])
        voxel_iscs.append(iscs)
    iscs = np.column_stack(voxel_iscs)
    
    # Summarize results (if requested)
    if summary_statistic == np.nanmean:
        iscs = np.tanh(np.nanmean(np.arctanh(iscs), axis=0))[np.newaxis, :]
    elif summary_statistic == np.mean:
        iscs = np.tanh(np.mean(np.arctanh(iscs), axis=0))[np.newaxis, :]
    elif summary_statistic == np.median:    
        iscs = summary_statistic(iscs, axis=0)[np.newaxis, :]
    elif not summary_statistic:
        pass
    else:
        raise ValueError("Unrecognized summary_statistic! Use None, np.median, or np.mean.")
    return iscs




In [4]:
# prepare spots
spot_list = ['commercial_aribnb_30s', 'commercial_att_30s', 'commercial_carscom_30s','commercial_cookies_30s',
             'commercial_dominos_30s', 'commercial_doritos_30s','commercial_expedia_30s','commercial_google_pixel_30s',
             'commercial_hr_block_30s','commercial_jersey_mikes_30s','commercial_lego_30s','commercial_meta_quest_2_30s',
             'commercial_milk_30s','commercial_progressive_30s','commercial_publix_30s','commercial_puma_30s',
             'commercial_starbucks_30s','commercial_under_armour_30s',
             'health_alcohol_30s','health_alzheimers_30s','health_covid_vaccine_30s','health_diet_30s',
             'health_drunk_driving_30s','health_fitness_30s','health_kidney_30s','health_mantherapy_30s',
             'health_prediabetes_30s','health_stroke_30s','health_vaping_30s','health_weight_30s']

n_spots = len(spot_list)

spot_list.sort()
print(len(spot_list))
print(spot_list[:3])

# prepare subject list
sub_folders = glob.glob("../data/00_raw_data/sub*")
sub_folders.sort()
subject_list = []
for f in sub_folders:
    subject_list.append(f[-6:])

n_subjects = len(subject_list)
print(n_subjects)
print(subject_list[:3])

condition_list = ['100', '50nd', '50wd']
n_conditions = len(condition_list)

30
['commercial_aribnb_30s', 'commercial_att_30s', 'commercial_carscom_30s']
59
['sub001', 'sub002', 'sub003']


In [5]:
stack_subs = subject_list * n_spots * n_conditions
stack_subs.sort()
print(len(stack_subs))

stack_conditions = condition_list * n_subjects * n_spots
print(len(stack_conditions))

stack_spots = spot_list  * n_conditions
stack_spots.sort()
stack_spots = stack_spots * n_subjects
print(len(stack_spots))


#create results df
df = pd.DataFrame(stack_subs, columns = ['subject'])
df['condition'] = stack_conditions
df['spot'] = stack_spots
df['recall'] = np.nan
df['recognition'] = np.nan
df['isc_ind2rest'] = np.nan
del stack_subs, stack_spots, stack_conditions
df

#df['condition'] = condition_list * n_subjects * n_spots
#df.loc[(df['condition'] == '100') & (df['subject'] == 'sub001')]
df

5310
5310
5310


,subject,condition,spot,recall,recognition,isc_ind2rest
0,sub001,100,commercial_aribnb_30s,NaN,NaN,NaN
1,sub001,50nd,commercial_aribnb_30s,NaN,NaN,NaN
2,sub001,50wd,commercial_aribnb_30s,NaN,NaN,NaN
3,sub001,100,commercial_att_30s,NaN,NaN,NaN
4,sub001,50nd,commercial_att_30s,NaN,NaN,NaN
...,...,...,...,...,...,...
5305,sub060,50nd,health_vaping_30s,NaN,NaN,NaN
5306,sub060,50wd,health_vaping_30s,NaN,NaN,NaN
5307,sub060,100,health_weight_30s,NaN,NaN,NaN
5308,sub060,50nd,health_weight_30s,NaN,NaN,NaN


In [6]:
x_length = 725

search_file_path = '../data/03_spots_memory_data/'


for current_spot_index in range(n_spots):
    current_spot = spot_list[current_spot_index]
    print(current_spot)

    for current_condition_index in range(n_conditions): 
        current_condition = condition_list[current_condition_index]
        print(current_condition)
        
        memory_pupil_data = []


        curr_search_folder = search_file_path + current_spot + '/' + current_condition + '/' 
        #print(curr_search_folder)

        memory_spot = glob.glob(os.path.join(curr_search_folder, '*recall*.csv'))
        memory_spot.sort()
        n_memory_spot = len(memory_spot)
        #print(n_memory_spot)
        
        memory_spot2 = glob.glob(os.path.join(curr_search_folder, '*recognition*.csv'))
        memory_spot2.sort()
        n_memory_spot2 = len(memory_spot2)
        #print(n_memory_spot2)

        if n_memory_spot2 != n_memory_spot:
            print('attention')
            
        
        #print('---')

        for curr_sub in range(n_memory_spot):

                    curr_file = memory_spot[curr_sub]
                    #print(curr_file)

                    curr_data =     pd.read_csv(curr_file)['pupil diameter']
                    curr_data_int = curr_data.interpolate(limit_direction='both' ).values[:x_length]
                    curr_data_int = curr_data_int - np.nanmean(curr_data_int[:300]) #subtract baseline
                    memory_pupil_data.append((curr_data_int[300:])) 
        this_spot_data = np.asarray(memory_pupil_data)

        print(this_spot_data.shape)

        arr = list(np.arange( n_memory_spot))
        #print(arr)

        for i, curr_sub in enumerate(arr):
            the_rest = arr[:i] + arr[i+1:]
            #print(curr_sub, the_rest)
            curr_sub_data = this_spot_data[curr_sub,:]
            other_data = np.mean(this_spot_data[the_rest, :], axis =0)
            sub = memory_spot[curr_sub][-18:-12]
            
            recall_status = int(memory_spot[curr_sub][-5:-4])
            recognition_status = int(memory_spot2[curr_sub][-5:-4])
            
            
            isc = np.corrcoef(curr_sub_data, other_data)[0,1]
            
            #'''
            print(sub)
            print(current_condition)
            print(current_spot)
            print('recall_status: {}'.format(recall_status) )
            print('recognition_status: {}'.format(recognition_status) )
            print(isc)
            print('---')#'''


            df.loc[(df['subject'] == sub) & (df['condition'] == current_condition) & (df['spot'] == current_spot), 'isc_ind2rest'] = isc  
            df.loc[(df['subject'] == sub) & (df['condition'] == current_condition) & (df['spot'] == current_spot), 'recall']       = recall_status  
            df.loc[(df['subject'] == sub) & (df['condition'] == current_condition) & (df['spot'] == current_spot), 'recognition']  = recognition_status  
            #print( df.loc[(df['subject'] == sub) & (df['condition'] == current_condition) & (df['spot'] == current_spot)].values)
            

df = df.dropna(axis=0, how='any', subset=['isc_ind2rest'])

df.head()

commercial_aribnb_30s
100
(20, 425)
sub001
100
commercial_aribnb_30s
recall_status: 0
recognition_status: 0
0.40475559496387953
---
sub002
100
commercial_aribnb_30s
recall_status: 0
recognition_status: 1
0.2705226531241173
---
sub007
100
commercial_aribnb_30s
recall_status: 0
recognition_status: 1
0.6727648672164781
---
sub008
100
commercial_aribnb_30s
recall_status: 0
recognition_status: 1
0.3471946593515282
---
sub013
100
commercial_aribnb_30s
recall_status: 1
recognition_status: 1
0.7473686255357995
---
sub014
100
commercial_aribnb_30s
recall_status: 1
recognition_status: 1
0.7516822339060725
---
sub019
100
commercial_aribnb_30s
recall_status: 1
recognition_status: 1
0.8948281483322207
---
sub020
100
commercial_aribnb_30s
recall_status: 1
recognition_status: 0
0.7600332476704318
---
sub025
100
commercial_aribnb_30s
recall_status: 0
recognition_status: 1
0.8234397850230152
---
sub026
100
commercial_aribnb_30s
recall_status: 1
recognition_status: 1
0.614684008161731
---
sub031
100
com

,subject,condition,spot,recall,recognition,isc_ind2rest
0,sub001,100,commercial_aribnb_30s,0.0,0.0,0.404756
4,sub001,50nd,commercial_att_30s,0.0,1.0,0.340053
6,sub001,100,commercial_carscom_30s,0.0,0.0,0.523108
9,sub001,100,commercial_cookies_30s,0.0,1.0,0.217574
12,sub001,100,commercial_dominos_30s,0.0,1.0,0.455560


In [60]:
df.to_csv('../data/03_spots_memory_data/all_subs_spots_conditions_memory_isc_reproduce.csv', index=False)


In [7]:
df.head()

,subject,condition,spot,recall,recognition,isc_ind2rest
0,sub001,100,commercial_aribnb_30s,0.0,0.0,0.404756
4,sub001,50nd,commercial_att_30s,0.0,1.0,0.340053
6,sub001,100,commercial_carscom_30s,0.0,0.0,0.523108
9,sub001,100,commercial_cookies_30s,0.0,1.0,0.217574
12,sub001,100,commercial_dominos_30s,0.0,1.0,0.455560


In [23]:
aggregated_df = df.groupby(['condition', 'spot'])[['recall','recognition', 'isc_ind2rest']].mean().reset_index()
aggregated_df.head()

,condition,spot,recall,recognition,isc_ind2rest
0,100,commercial_aribnb_30s,0.55,0.85,0.685177
1,100,commercial_att_30s,0.00,1.00,0.487505
2,100,commercial_carscom_30s,0.25,0.80,0.615673
3,100,commercial_cookies_30s,0.10,0.95,0.692899
4,100,commercial_dominos_30s,0.30,1.00,0.616790


In [24]:
aggregated_df.to_csv('../data/03_spots_memory_data/aggregated_df_reproduce.csv', index=False)
